In [ ]:
import numpy as np
import os ## added
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' ## added
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import functools
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'


# Lime model for explaination

model == cnn model
X == pixel map
y ==  labels correspond to each map
m == at what stride the resampling frames will be chosen for model explaination

### for more detail on the lime protocol: https://towardsdatascience.com/how-to-explain-image-classifiers-using-lime-e364097335b4

In [ ]:
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime.wrappers.scikit_image import SegmentationAlgorithm
from lime import lime_image
import numpy as np


def cnn_lime(model, X, y, m): ## m = stride

    explainer = lime_image.LimeImageExplainer(verbose = False)
    segmenter = SegmentationAlgorithm('slic', n_segments=100, compactness=1, sigma=1)
    # =============================================================================

    # =============================================================================

    n_features = 100
    n_samples = 1000

    importance_pic = []
    size = len(X[0])
#     n_lime = int(len(y)/10)
    n_lime = int(len(y)/m)
    for i in range(size):
        line = []
        for j in range(size):
            line.append(0)
        importance_pic.append(line)

    for i in range(n_lime):
        print(i)
#             explanation = explainer.explain_instance(X[i*10].astype('float64'),
        explanation = explainer.explain_instance(X[i*m].astype('float64'),
                                                # classifier_fn = model[j].predict_proba,  
                                                 classifier_fn = (model.predict), ## added
                                                 top_labels=2, hide_color=0, num_samples=n_samples, segmentation_fn=segmenter)
        temp, mask = explanation.get_image_and_mask(y[i*m], positive_only=True, num_features=n_features, hide_rest=False)

        importance_pic = np.array(importance_pic) + np.array(mask)
            
#             if(sum(sum(mask)))!=0:
#                 mask = [x/(sum(sum(mask))) for x in mask]
#                 importance_pic = np.array(importance_pic) + np.array(mask)


    # =============================================================================
    importance_pic = importance_pic / n_lime ## added 
    return importance_pic, explainer, segmenter



## Load traj and pdb (Trajectory will be read using mdanalysis. So technically dcd/xtc should also work)
For trajectory load related modifications, check traj_utils.py (function: traj_pre, load_traj, read_traj)
The CNN modelling function now only do a binary classification. But we can extend it as per our need.

In [ ]:
## List of systems
file0_1 = "../../crRNA_R1-3_notREClobes.nc"
file0_2 = "../../crRNA_R1-3_notREClobes.pdb"
file1_1 = "../../tgRNA_R1-3_notREClobes.nc"
file1_2 = "../../tgRNA_R1-3_notREClobes.pdb"
stride = 5
# if_prt = 1
# if_save = 1
# if_dtl = 1

In [ ]:
try:
    from . import traj_utils
except Exception:
    import traj_utils
_, y_all, X_all = traj_utils.traj_pre(file0_1, file1_1, file0_2, file1_2, stride)
#_, X_all, y_all = traj_utils.traj_pre(file0_1, file1_1, file0_2, file1_2)
print("Preprocess Done.\n")

## y_all: labels [0/1]
## X_all: pixel_maps

In [ ]:
print("length of y_all:",len(y_all))
print("length of X_all:",len(X_all))
# print(y_all[27990:])
# print(X_all[1].shape)
# X_1 = X_all[1].reshape(1,X_all[1].shape[0],X_all[1].shape[1],3)
# print(X_1.shape)

## Split the data to train/test set
K: k-fold cross-validation

group: Divide each traj into 'group' number of groups in time order. Each group then randomly divided into 5 folds. take 4 fold from each group to make the training set and 1 fold for the validation set

cap: population in each group


In [ ]:
try:
    from . import split_data
except Exception:
    import split_data
group = 10
cap = int(len(X_all) / group)
k = 5
X_train, y_train, X_test, y_test = split_data.split_by_group(X_all, y_all, group, cap, k)

In [ ]:
# print(len(X_train))

In [ ]:
# print(len(X_train[3]))

## Categorical data encoding

In [ ]:
try:
    from . import data_utils
except Exception:
    import data_utils
num_classes = 2
X_train, y_train, X_test, y_test = data_utils.data_encode(X_train, y_train, X_test, y_test, k, num_classes)


In [ ]:
# print(y_train[0][:10])
# print(y_train[4][:10])
# if (y_train[1] == y_train[4]).all():
#     print("fuck")

## Train CNN
batch_size: 32/64/128 should be fine.. may vary with number of inputs

epochs: not less than 5

In [ ]:
try:
    from . import cnn_model
except Exception:
    import cnn_model

batch_size = 32
epochs = 10
model = cnn_model.cnn_build(X_train, k, num_classes)
model, history = cnn_model.cnn_train(model, X_train, y_train, X_test, y_test, k, batch_size, epochs, if_dtl)

In [ ]:
## Evaluate your model
try:
    from . import model_evaluate
except Exception:
    import model_evaluate

model_evaluate.eva_cross_acc_all(model, X_train, X_test, y_train, y_test)


In [ ]:
## Write your model
for t in range(k):
        model[t].save('model' + str(t) + '.h5')


In [ ]:
# print(len(model))

In [ ]:
# print((model[1].predict(X_train[1])>0.5).astype("int32"))
# print(y_train[1][1])


In [ ]:
# y_train_pre  = (model.predict(X_1))
# print(y_train_pre)

## model explaination using LIME

In [ ]:
from keras.models import load_model
importance_pic1 = []
size = len(X_all[0])
for i in range(size):
    line = []
    for j in range(size):
        line.append(0)
    importance_pic1.append(line)

for i in list(range(5)): ## how many models you have created by k-fold CV
    model = load_model(f"model{i}.h5")
    importance_pic, explainer, segmenter = cnn_lime(model, X_all, y_all, 100)
    print(importance_pic1)
    print(f"model{i} done.....")
#     print(len(importance_pic1))
    importance_pic1 = np.array(importance_pic1) + np.array(importance_pic)



# Write the atom score xlsx file

In [ ]:
import xlsxwriter

workbook = xlsxwriter.Workbook('atom.xlsx')
worksheet = workbook.add_worksheet()
size = len(importance_pic)
# n_lime = int(len(y)/m)
for i in range(size):
    for j in range(size):
        worksheet.write(i * size + j, 0, i * size + j + 1)
        worksheet.write(i * size + j, 1, importance_pic1[i][j])
workbook.close()



# Read the xlsx file and map the scores from atom to residue

In [ ]:
f = open("../../crRNA_R1-3_notREClobes.pdb", "r")

data = f.readlines()
f.close()
p = len(data) - 1
data.pop(p)
data.pop(p - 1)
data.pop(0)


import pandas as pd
bond = []
for line in data:
    bond_line = line.split(' ')
    while '' in bond_line:
        bond_line.remove('')
    bond.append([int(bond_line[1]), int(bond_line[4])])
df1 = pd.DataFrame(bond) # added


### ----- Added to read xlsx --- ###
import pandas as pd
import openpyxl
df2 = pd.read_excel(
  "atom.xlsx", header=None,
  engine='openpyxl',
)
 ### ------------------------------------####
df1[2] = df2[1] ## added
df3 = df1.drop([0], axis=1) ## added
df4 = df3.groupby(by=[1], group_keys=False).mean() ## added
df5 = df4.sort_values(by=df4.columns[0], ascending=False) ## added
df5.to_csv(r'res_score_old.dat', header=None,  sep=' ', mode='a')


In [ ]:
# print(df2)